In [ ]:
from functions.df_alias import alias
from functions.df_produtos import df_produto_final
from functions.df_veiculos import df_veiculo_final
from functions.connect_api import consumir_api
from functions.bigquery import write_bigquery, credencial_bq
from datetime import datetime
import pandas as pd
import pandas_gbq

In [ ]:
#consumindo api e salvando no df
veiculos = consumir_api()
df_veiculos = df_veiculo_final(veiculos)

df_alias = alias()

df_produtos = df_produto_final()

## Realizando o Join entre a tabela de produtos e a tabela de veículos através da tabela de alias

In [ ]:
df_produtos_alias = pd.merge(df_produtos, df_alias, left_on='Código', right_on='Código Fabricante')
df = pd.merge(df_produtos_alias, df_veiculos, left_on='Alias', right_on='Código Fabricante')

## Script na qual remove caracteres especiais, letras maiusculas e acentuações das colunas, para que o bigquery não tenha problemas para escrever o df em tabela SQL

In [ ]:
from unidecode import unidecode
header_sem_acento = {i : unidecode(i.lower().replace('(','').replace(')','').replace(' ','_'))\
                     for i in list(df.columns)}
    #passando o dicionário para corrigir os nomes adequadamente
df = df.rename(columns=header_sem_acento)

## Inserindo colunas solicitadas no desafio

In [ ]:
df['dt_insert'] = datetime.now()
df['candidate_name']= 'Grégory Pierroti'

## Escrevendo DF no bigQuery

In [ ]:
param = write_bigquery()
tabela = write_bigquery()[0]
project_id = write_bigquery()[1]

In [ ]:
df.to_gbq(destination_table=tabela,project_id=project_id, credentials=credencial_bq(),if_exists='replace')